# Wikipedia Notable Life Expectancies
# [Notebook 8: Data Cleaning Part 7](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean7_thanak_2022_07_26.ipynb)
### Context

The
### Objective

The
### Data Dictionary
- Feature: Description

### Importing Libraries

In [1]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
import sqlite3 as sql

# To save/open python objects in pickle file
import pickle

# To help with reading, cleaning, and manipulating data
import pandas as pd
import numpy as np
import re

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

# To play auditory cue when cell has executed, has warning, or has error and set chime theme
import chime

chime.theme("zelda")

<IPython.core.display.Javascript object>

## Data Overview

### Reading, Sampling, and Checking Data Shape

In [2]:
# Reading the dataset
conn = sql.connect("wp_life_expect_clean6.db")
data = pd.read_sql("SELECT * FROM wp_life_expect_clean6", conn)

# Making a working copy
df = data.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 98060 rows and 38 columns.


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
0,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21,1994,January,None,None,dancer,ballet designer and director,None,None,None,None,None,None,None,None,86.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,3.091042,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12,1994,January,None,None,economist,writer,and academic,None,None,None,None,None,None,None,68.0,None,Ireland,None,None,2.564949,0,0,0,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [3]:
# Checking last 2 rows of the data
df.tail(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
98058,9,Aamir Liaquat Hussain,", 50, Pakistani journalist and politician, MNA .",https://en.wikipedia.org/wiki/Aamir_Liaquat_Hussain,99,2022,June,", since",None,,MNA,None,None,None,None,None,None,None,None,50.0,None,Pakistan,None,"2002 2007, since 2018",4.605170,0,0,0,0,0,1,0,0,1,0,0,0,2
98059,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3,2022,June,None,None,engineer,member of the Academy of Engineering,None,None,None,None,None,None,None,None,86.0,None,"China, People's Republic of",None,None,1.386294,0,0,0,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [4]:
# Checking a sample of the data
df.sample(5)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
69373,7,Al Capps,", 79, American record producer, arranger and composer.",https://en.wikipedia.org/wiki/Al_Capps,5,2018,June,None,None,record producer,arranger and composer,None,None,None,None,None,None,None,None,79.0,None,United States of America,None,None,1.791759,0,0,0,0,0,0,0,0,0,0,0,0,0
69476,13,Tom Gear,", 69, American politician, member of the Virginia House of Delegates .",https://en.wikipedia.org/wiki/Tom_Gear,15,2018,June,,None,,member of the Virginia House of Delegates,None,None,None,None,None,None,None,None,69.0,None,United States of America,None,2002 2010,2.772589,0,0,0,0,0,0,0,0,1,0,0,0,1
57812,8,Elizabeth Roemer,", 87, American astronomer.",https://en.wikipedia.org/wiki/Elizabeth_Roemer,8,2016,April,None,None,astronomer,None,None,None,None,None,None,None,None,None,87.0,None,United States of America,None,None,2.197225,0,0,0,0,0,0,0,0,0,0,0,0,0
34207,29,Jim Seymour,", 64, American football player .",https://en.wikipedia.org/wiki/Jim_Seymour_(American_football),5,2011,March,Chicago Bears,None,,None,None,None,None,None,None,None,None,None,64.0,None,United States of America,None,Chicago Bears,1.791759,0,0,0,0,0,0,1,0,0,0,0,0,1
40341,20,Jaouad Akaddar,", 28, Moroccan footballer, cardiac arrest.",https://en.wikipedia.org/wiki/Jaouad_Akaddar,4,2012,October,None,None,,cardiac arrest,None,None,None,None,None,None,None,None,28.0,None,Morocco,None,None,1.609438,0,0,0,0,0,0,1,0,0,0,0,0,1


<IPython.core.display.Javascript object>

### Checking Data Types, Duplicates, and Null Values

In [5]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98060 entries, 0 to 98059
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   day                        98060 non-null  object 
 1   name                       98060 non-null  object 
 2   info                       98060 non-null  object 
 3   link                       98060 non-null  object 
 4   num_references             98060 non-null  int64  
 5   year                       98060 non-null  int64  
 6   month                      98060 non-null  object 
 7   info_parenth               36661 non-null  object 
 8   info_1                     22 non-null     object 
 9   info_2                     98028 non-null  object 
 10  info_3                     48896 non-null  object 
 11  info_4                     10264 non-null  object 
 12  info_5                     1265 non-null   object 
 13  info_6                     181 non-null    obj

<IPython.core.display.Javascript object>

#### Observations:
- With our dataset loaded, we can pick up where we left off with extracting known_for values by rebuilding `known_for_dict`.

### Extracting `known_for` Continued

#### Finding `known_for` Roles in `info_2`

In [10]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [49]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [48]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "Catholic prelate" in df.loc[index, "info"]],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [47]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [50]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "defrocked" in df.loc[index, "info"]]]

<IPython.core.display.Javascript object>

In [ ]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [
#         index
#         for index in df.index
#         if "outlaw country music singer songwriter" in df.loc[index, "info"]
#     ]
# ]

In [51]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "defrocked Catholic prelate"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [52]:
# Creating lists for each category
politics_govt_law = [
    "Patriotic",
]

arts = []
sports = []
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = [
    "Syro Malabar Catholic prelate",
    "Eastern Catholic prelate",
    "clandestine Catholic prelate",
    "Old Catholic prelate",
    "Catholic prelate and theologian",
    "Catholic prelate and first cardinal",
    "Maronite Catholic prelate",
    "Catholic prelate and Cardinal",
    "Coptic Catholic prelate",
    "Catholic prelate and bishop",
    "Catholic prelate and cardinal",
    "Catholic prelate and",
    "and Catholic prelate",
    "Catholic prelate",
]
social = []
crime = [
    "defrocked",
]
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [53]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [54]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['spiritual'] ==1].sample(2)

CPU times: total: 8.34 s
Wall time: 8.32 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
53450,27,Peter Celestine Elampassery,", 76, Indian Roman Catholic prelate, Bishop of Jammu-Srinagar .",https://en.wikipedia.org/wiki/Peter_Celestine_Elampassery,9,2015,May,,None,,Bishop of Jammu Srinagar,None,None,None,None,None,None,None,None,76.0,None,India,Italy,1998 2014,2.302585,0,0,1,0,0,0,0,0,0,0,0,0,1
63852,2,Leon Lemmens,", 63, Belgian Roman Catholic prelate, Auxiliary Bishop of Mechelen-Brussels , leukemia.",https://en.wikipedia.org/wiki/Leon_Lemmens,10,2017,June,since,None,,Auxiliary Bishop of Mechelen Brussels,leukemia,None,None,None,None,None,None,None,63.0,None,Belgium,Italy,since 2011,2.397895,0,0,1,0,0,0,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [55]:
#### Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 44902 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [56]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [263]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [261]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "physicist" in df.loc[index, "info"]], "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [259]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [260]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [index for index in df.index if "physicist and science" in df.loc[index, "info"]]
# ]

<IPython.core.display.Javascript object>

In [ ]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [
#         index
#         for index in df.index
#         if "outlaw country music singer songwriter" in df.loc[index, "info"]
#     ]
# ]

In [264]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "health physicist"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [265]:
# Creating lists for each category
politics_govt_law = ["arms control expert", "refusenik"]

arts = []
sports = []
sciences = [
    "physicist and group leader in the Manhattan Project",
    "physicist and molecular biologist",
    "physicist and computer research executive",
    "Nobel Prize winning biophysicist",
    "physicist and grandson of Yuan Shikai",
    "physicist and polymer researcher",
    "physicist and hydrodynamicist",
    "nuclear physicist and inventor",
    "physicist and a leader in controlled fusion research",
    "computational physicist and the father of plasma based acceleration techniques",
    "biophysicist and theoretical ecologist",
    "physicist and co winner of Nobel Prize in Physics in",
    "leading physicist in the study of waves",
    "physicist and Nobel laureate",
    "physicist ane engineer",
    "astrophysicist and radio astronomer",
    "physicist and artificial intelligence pioneer",
    "physicist and civil engineer",
    'physicist who coined the term "black hole"',
    "physicist who co discovered the Wigner Seitz cell",
    "physicist and former director of SLAC",
    "physicist at Uppsala University",
    "physicist who won the Nobel Prize for Physics in",
    "physicist who built the first laser",
    "physicist who was a pioneer of solid state physics",
    "molecular biophysicist and crystallographer",
    "physicist and member of the Manhattan Project",
    "physicist and color scientist",
    "marine geologist and geophysicist",
    "theoretical physicist and astronomer",
    "physicist and electronics engineer",
    "physicist and inventor of the first digital computer",
    "pioneering biophysicist and virologist",
    "physicist Nobel Prize in Physics laureate",
    "theoretical physicist and magneto ionic theory pioneer",
    "theoretical physicist and Nobel Prize laureate",
    "physicist and jet engine designer",
    "geophysicist and oceanographer",
    "physicist and winner of the Nobel Prize in Physics",
    "physicist known for the Casimir effect",
    "nuclear physicist who worked at the Manhattan Project Metallurgical Laboratory",
    "physicist and team member of the Manhattan Project",
    "biophysicist and biochemist",
    "experimental physicist and scientist",
    "physicist and radiation health physics pioneer",
    "physicist and co inventor of the laser with Charles Townes",
    "nuclear engineer and physicist",
    'physicist known as "the father of Pulsed Power"',
    "physicist and physical chemist",
    "chemist and nuclear physicist",
    "physicist and recipient of the Nobel Prize in Physics",
    "differential geometer and mathematical physicist",
    "physicist and statistician",
    "physicist and father of Joan Baez and Mimi Fariña",
    "nuclear physicist and engineer",
    "physicist and microbiologist",
    "nuclear physicist and ufologist",
    "chemist and biophysicist",
    "condensed matter physicist",
    "physicist and researcher",
    "geophysicist and structural geologist",
    "mesoscopic physicist",
    "physicist and specialist in solid state laser",
    "physicist and aircraft designer",
    "physicist specialized in theoretical catalysis",
    "biologist and biophysicist",
    "thermal physicist",
    "atomic physicist",
    "biophysicist and science",
    "research physicist",
    "theoretical physicist and nuclear engineer",
    "neurophysicist",
    "experimental nuclear physicist",
    "health physicist",
    "physicist and parapsychologist",
    "physicist and skeptic",
    "solid state physicist",
    "biophysicist and virologist",
    "atmospheric physicist",
    "physicist and geneticist",
    "electrical engineer and physicist",
    "climate physicist",
    "nuclear and particle physicist",
    "explosives engineer and physicist",
    "physicist and neurobiologist",
    "mathematical physicist and cosmologist",
    "metallurgist and physicist",
    "mathematical geophysicist and seismologist",
    "East physicist",
    "theoretical physicist and astrophysicist",
    "Nobel Prize winning physicist",
    "optical physicist",
    "metal physicist",
    "metal and detonation physicist",
    "solar physicist",
    "oceanographic physicist",
    "geophysicist and planetary scientist",
    "astroparticle physicist",
    "accelerator physicist",
    "engineer and physicist",
    "molecular biophysicist",
    "physicist and radio astronomer",
    "physicist and meteorologist",
    "physicist and computer scientist",
    "astronomer and physicist",
    "physicist and chemist",
    "chemical physicist",
    "physicist and electrical engineer",
    "physicist and astronomer",
    "astronomer and astrophysicist",
    "medical physicist",
    "space physicist",
    "plasma physicist",
    "chemist and physicist",
    "physicist and inventor",
    "experimental physicist",
    "physicist and engineer",
    "mathematical physicist",
    "particle physicist",
    "biophysicist",
    "geophysicist",
    "and nuclear physicist",
    "nuclear physicist and",
    "nuclear physicist",
    "and astrophysicist",
    "astrophysicist and",
    "astrophysicist",
    "and theoretical physicist",
    "theoretical physicist and",
    "theoretical physicist",
    "physicist and",
    "and physicist",
    "physicist",
]

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [266]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [ ]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['sciences'] ==1].sample(2)

#### Checking the Number of Rows without a First Category

In [ ]:
#### Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

In [ ]:
print("dunzo!")

# Sound notification when cell executes
chime.success()

#### Finding `known_for` Roles in `info_2`

In [ ]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

In [ ]:
# Code to check each value
roles_list.pop()

In [ ]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "writer" in df.loc[index, "info"]], "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

In [ ]:
# # Code to check each specific value
# specific_roles_list.pop()

In [ ]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "and science writer" in df.loc[index, "info"]]]

In [ ]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [
#         index
#         for index in df.index
#         if "outlaw country music singer songwriter" in df.loc[index, "info"]
#     ]
# ]

In [ ]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "outlaw country music singer songwriter"]

#### Creating Lists for Each `known_for` Category

In [ ]:
# Creating lists for each category
politics_govt_law = []

arts = []
sports = [


]
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [ ]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

#### Extracting Category from `info_2`

In [ ]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['arts'] ==1].sample(2)

#### Checking the Number of Rows without a First Category

In [ ]:
#### Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Observations:
- It is time to export our dataframe and start a new notebook.

### Exporting Dataset to SQLite Database [wp_life_expect_clean7.db]()

In [ ]:
# # Exporting dataframe

# # Saving dataset in a SQLite database
# conn = sql.connect("wp_life_expect_clean7.db")
# df.to_sql("wp_life_expect_clean7", conn, index=False)

# # Chime notification when cell executes
# chime.success()

# [Proceed to Data Cleaning Part 8 ]()